<a href="https://colab.research.google.com/github/hjn14133/Machine-Learning/blob/main/Bagging_Regression_Yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn import ensemble, datasets, tree
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import sys
import warnings

warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("FT_join_JH_cleaned.csv")

X = df[['DISTANCE', 'Heading','Elevation', 'AppliedRat']]
y = df['VRYIELDBAL']

# y = df['Loan_Rat']

In [4]:
# Perform data splitting using 80/20 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [5]:
from sklearn.ensemble import BaggingRegressor

bag_regressor = BaggingRegressor(random_state=1)
bag_regressor.fit(X_train, Y_train)

BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=1, verbose=0,
                 warm_start=False)

In [6]:
Y_preds = bag_regressor.predict(X_test)

print(Y_preds[:10])
print(Y_test[:10])

print('Training Coefficient of R^2 : %.3f'%bag_regressor.score(X_train, Y_train))
print('Test Coefficient of R^2 : %.3f'%bag_regressor.score(X_test, Y_test))

bag_regressor.get_params()

[2.82717383 1.64773207 3.44802164 2.88678295 3.17925313 3.92721023
 1.28796147 0.99366359 3.66777374 1.70646882]
4755     2.650791
11830    1.178584
1980     3.350966
3036     3.002728
853      3.852232
2028     3.329378
12419    0.677427
10144    1.625142
1889     3.387610
11323    1.367832
Name: VRYIELDBAL, dtype: float64
Training Coefficient of R^2 : 0.936
Test Coefficient of R^2 : 0.655


{'base_estimator': None,
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1,
 'verbose': 0,
 'warm_start': False}

In [7]:
# Calculate the absolute errors
errors = abs(Y_preds - Y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), '.')


# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / Y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 0.4 .
Accuracy: 72.02 %.


In [9]:
# get importance
importance = bag_regressor.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

AttributeError: ignored

In [17]:
params = {'base_estimator': [None, LinearRegression(), KNeighborsRegressor()],
          'n_estimators': [5, 10, 100, 500, 1000],
          #'max_samples': [1.0, 2],
          'max_features': [0.5, 1.0, 2],
          'bootstrap': [True],
          'bootstrap_features': [False]}

bagging_regressor_grid = GridSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), param_grid =params, cv=3, n_jobs=-1, verbose=1)
bagging_regressor_grid.fit(X_train, Y_train)

print('Train R^2 Score : %.3f'%bagging_regressor_grid.best_estimator_.score(X_train, Y_train))
print('Test R^2 Score : %.3f'%bagging_regressor_grid.best_estimator_.score(X_test, Y_test))
print('Best R^2 Score Through Grid Search : %.3f'%bagging_regressor_grid.best_score_)
print('Best Parameters : ',bagging_regressor_grid.best_params_)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  6.4min finished


Train R^2 Score : 0.956
Test R^2 Score : 0.670
Best R^2 Score Through Grid Search : 0.657
Best Parameters :  {'base_estimator': None, 'bootstrap': True, 'bootstrap_features': False, 'max_features': 1.0, 'n_estimators': 1000}


In [21]:
Y_preds_grid = bagging_regressor_grid.best_estimator_.predict(X_test)

# Calculate the absolute errors
error_grid = abs(Y_preds_grid - Y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(error_grid), 2), '.')


# Calculate mean absolute percentage error (MAPE)
mape_grid = 100 * (error_grid / Y_test)
# Calculate and display accuracy
accuracy_grid = 100 - np.mean(mape_grid)
print('Accuracy:', round(accuracy_grid, 2), '%.')

Mean Absolute Error: 0.37 .
Accuracy: 74.3 %.
